ReadMe
---



---

Sahabat Asisten Belajar (SABA) or study companion is our capstone project for Bangkit 2021 For more information please visit or git hub [SABA](https://github.com/Bagoes-Heikhal/SABA)

In [4]:
# Chatbot with Deep NLP
#Importing the libraries
import numpy as np
import tensorflow
import re
import time 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Bidirectional
from google.colab import drive

In [34]:
print(tensorflow.__version__)

2.5.0


In [5]:
drive.mount("/content/drive")

Mounted at /content/drive


**Data Preprocess**
---


---



In [8]:
############## Import Dataset #############
# questions = open ('./QuestionFisika.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
# answers = open ('./AnswerFisika.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

############## Untuk Drive ################## 
questions = open ('/content/drive/MyDrive/Colab Notebooks/SABA/data/QuestionFisika.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
answers = open ('/content/drive/MyDrive/Colab Notebooks/SABA/data/AnswerFisika.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [9]:
####### Cleaning text dari tanda baca dan agar seragam #######
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r'" - "+', "", text)
    text = re.sub(r'" +"', " ", text)
    text = re.sub(r'"^ "', "", text)
    text = re.sub(r'[-()\"#/@;:<>{}`+=~|.!?,]', "", text)
    text = re.sub(r'[\-\-]', "", text)
    text = re.sub(r'\.\.\.', "", text)
    text = re.sub(r'^- ', "", text)
    return text

In [10]:
############### Petanyaan dan Jawaban clean ###############
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [11]:
############### Untuk Tokenizer dan input embedding layers ############### 
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

print(len(word2count))

840


In [12]:
############ Membatasi batas minimum dictionary yang digunakan ##############
threshold = 5

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= threshold:
        vocab[word] = word_num
        word_num += 1

print(len(vocab))

840


In [13]:
##### Memberikan Token Start of Sentence (SOS),
##### End of Sentence (EOS)
##### PAD untuk padding dan OUT untuk kata yang tidak ada di vocab

for i in range(len(clean_answers)):
    clean_answers[i] = '<SOS> ' + clean_answers[i] + ' <EOS>'

tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

    
print(clean_questions[0])
print(clean_answers[0])

jangka sorong
<SOS> jangka sorong adalah alat ukur yang mampu mengukur jarak kedalaman maupun ‘diameter dalam’ suatu objek dengan tingkat akurasi dan presisi yang sangat baik ±005 mm <EOS>


In [14]:
###### inverse untuk decode ######
inv_vocab = {w:v for v, w in vocab.items()}

In [15]:
###### Encoder Input ######
encoder_inp = []
for line in clean_questions:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

In [16]:
###### Decoder Input ######
decoder_inp = []
for line in clean_answers:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

**Encode Model**
---

---





In [17]:
############ Hyperparameters ##################
max_input_len = 15
lstm_layers = 400
VOCAB_SIZE = len(vocab)

In [18]:
#### Padding agar panjang encoder dan decoder sama ketika di training ####
encoder_inp = pad_sequences(encoder_inp, maxlen = max_input_len, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, maxlen = max_input_len, padding='post', truncating='post')

In [19]:
#### Untuk output target learning ####

decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:])

decoder_final_output = pad_sequences(decoder_final_output, maxlen = max_input_len, padding='post', truncating='post')
decoder_final_output = to_categorical(decoder_final_output, len(vocab))
print(decoder_final_output.shape)

(3728, 15, 844)


In [20]:
##### Embedding layer utama ####
embedding = Embedding(VOCAB_SIZE + 1,
                      output_dim = 50, 
                      input_length = max_input_len,
                      trainable = True,
                      mask_zero = True 
                      )

In [21]:
########### Encode  #############

########### Encoder #############
e_input = Input(shape=(max_input_len, ))
e_embeded = embedding(e_input)
e_lstm = LSTM(lstm_layers, return_sequences=True, return_state=True)
e_op, h_state, c_state = e_lstm(e_embeded)
e_states = [h_state, c_state]

########### Decoder #############
d_input = Input(shape=(max_input_len, ))
d_embeded = embedding(d_input)
d_lstm = LSTM(lstm_layers, return_sequences=True, return_state=True)
d_output, _, _ = d_lstm(d_embeded, initial_state = e_states)
d_dense = Dense(VOCAB_SIZE, activation='softmax')
d_outputs = d_dense(d_output)

model = Model([e_input, d_input], d_outputs)

**TRAINING**
---

---






In [32]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['acc'])

# from keras.utils.vis_utils import plot_model
# plot_model(model, show_shapes=True, show_layer_names=True)

In [26]:
############################ Untuk Memulai Training ############################

# model.fit([encoder_inp, decoder_inp], decoder_final_output, epochs=50, validation_split=0.2)

**Inference**
---



---



In [28]:
######### Decode  #############

######### Encoder #############
enc_model = Model([e_input], e_states)

######### Decoder #############
d_input_h = Input(shape=(lstm_layers,))
d_input_c = Input(shape=(lstm_layers,))
d_states_inputs = [d_input_h, d_input_c]
d_outputs, d_state_h, d_state_c = d_lstm(d_embeded, initial_state = d_states_inputs)
d_states = [d_state_h, d_state_c]
dec_model = Model([d_input] + d_states_inputs, [d_outputs] + d_states)

In [ ]:
############# Untuk Meload Model & weigth ##################
# enc_model.load_weights('./enc_model_fisika.h5')
# dec_model.load_weights('./dec_model_fisika.h5')
# model.load_weights('./main_model.h5')

In [33]:
############ plot Model ############

# from keras.utils.vis_utils import plot_model
# plot_model(enc_model, show_shapes=True, show_layer_names=True)
# plot_model(dec_model, show_shapes=True, show_layer_names=True)
# enc_model.summary()
# dec_model.summary()

In [ ]:
########## Untuk mengholah data input sebelum di inference ##########

def input_sentence(text):
    user_ = clean_text(text)
    user = [user_]

    inp_sentence = []
    for sentence in user:
        lst = []
        for y in sentence.split():
            try:
                lst.append(vocab[y])
            except:
                lst.append(vocab['<OUT>'])
        inp_sentence.append(lst)
    
    inputs_sentence = pad_sequences(inp_sentence, max_input_len, padding='post')
    states_value = enc_model.predict(inputs_sentence)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = vocab['<SOS>']

    return target_seq, states_value

In [ ]:
############# BOT SABA #################
user_ = ""
while user_ != 'quit':
    user_  = input("you : ")
    target_seq, states_value = input_sentence(user_)

    stop_condition = False
    decoded = ''

    while not stop_condition :
        output_tokens , h, c= dec_model.predict([target_seq] + states_value )
        input_tokens = d_dense(output_tokens)
        word_index = np.argmax(input_tokens[0, -1, :])

        ### Invers dari angka ke word index dalam vocab ###
        word = inv_vocab[word_index] + ' '
        if word != '<EOS> ':
            decoded += word  
        if word == '<EOS> ' or len(decoded.split()) > max_input_len:
            stop_condition = True 

        target_seq = np.zeros((1 , 1))  
        target_seq[0 , 0] = word_index
        states_value = [h, c]

    print("user :", user_)
    print("chatbot attention : ", decoded)
    print("==============================================")  

**Saving Model & Weights**
---

---

In [ ]:
# enc_model.save_weights('./enc_model_fisika.h5')
# dec_model.save_weights('./dec_model_fisika.h5')
# model.save_weights('./main_model.h5')